In [1]:
import pandas as pd
from src.StreamPort.machine_learning.analyses import MachineLearningAnalyses

train_data = pd.read_csv("dev/train_features.csv")
train_metadata = pd.read_csv("dev/train_metadata.csv")
ana = MachineLearningAnalyses(variables = train_data, metadata = train_metadata)
print(ana)
ana.plot_data()

c:\Users\Sandeep\Desktop\StreamPort\sp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



MachineLearningAnalyses 
  variables: 18 rows, 67 columns
  metadata: 18 rows, 14 columns



In [2]:
from src.StreamPort.machine_learning.methods import MachineLearningMethodIsolationForestSklearn
from src.StreamPort.machine_learning.methods import MachineLearningScaleFeaturesScalerSklearn

scl = MachineLearningScaleFeaturesScalerSklearn()

iso = MachineLearningMethodIsolationForestSklearn(random_state = 30, contamination = "auto")

#scales and fits the model and upgrades analyses class to IsolationForestAnalyses
ana = scl.run(ana)
ana = iso.run(ana)
print(ana.__class__)

ana.plot_data()

<class 'src.StreamPort.machine_learning.analyses.IsolationForestAnalyses'>


In [3]:
ana.train()
fig_train_scores=ana.plot_scores()
fig_train_scores.write_image("dev/figures/fig_train_scores.png", width=1100, height= 350, scale = 3)
fig_train_scores.show()

In [4]:
test_data = pd.read_csv("dev/test_features.csv")
test_metadata = pd.read_csv("dev/test_metadata.csv")

ana.predict(test_data, test_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)

fig_test_scores = ana.plot_scores()
fig_test_scores.write_image("dev/figures/fig_test_scores.png", width=1100, height= 350, scale = 3)
fig_test_scores.show()

    index  threshold     score  confidence    class
0     116  -0.046124 -0.170223        3.69  outlier
1     117  -0.046124 -0.059685        1.29  outlier
2     118  -0.046124 -0.022119        0.48   normal
3     119  -0.046124 -0.033691        0.73   normal
4     120  -0.046124 -0.019847        0.43   normal
5     121  -0.046124 -0.021224        0.46   normal
6     122  -0.046124 -0.032316        0.70   normal
7     123  -0.046124 -0.019155        0.42   normal
8     124  -0.046124 -0.027021        0.59   normal
9     125  -0.046124 -0.013025        0.28   normal
10    126  -0.046124 -0.037094        0.80   normal
11    127  -0.046124 -0.036929        0.80   normal
12    128  -0.046124 -0.031038        0.67   normal
13    129  -0.046124 -0.041049        0.89   normal
14    130  -0.046124 -0.007637        0.17   normal
15    131  -0.046124 -0.031494        0.68   normal
16    132  -0.046124 -0.031852        0.69   normal


In [5]:
#ana.add_prediction()
#print(ana)

In [6]:
fig_test_features = ana.plot_data()
fig_test_features.write_image("dev/figures/fig_test_features.png", width=1100, height= 350, scale = 3)
fig_test_features.show()

In [7]:
test2_data = pd.read_csv("dev/test2_features.csv")
test2_metadata = pd.read_csv("dev/test2_metadata.csv")
test_samples = test2_metadata["index"]

ana.predict(test2_data, test2_metadata)
outliers_test = ana.test_prediction_outliers()
print(outliers_test)
ana.plot_scores()

    index  threshold     score  confidence    class
0     266  -0.046124 -0.081019        1.76  outlier
1     267  -0.046124 -0.026941        0.58   normal
2     268  -0.046124 -0.082643        1.79  outlier
3     269  -0.046124 -0.063122        1.37  outlier
4     270  -0.046124 -0.040936        0.89   normal
5     271  -0.046124 -0.051454        1.12  outlier
6     272  -0.046124 -0.055149        1.20  outlier
7     273  -0.046124 -0.054858        1.19  outlier
8     274  -0.046124 -0.042529        0.92   normal
9     275  -0.046124 -0.052742        1.14  outlier
10    276  -0.046124 -0.070863        1.54  outlier
11    277  -0.046124 -0.044016        0.95   normal
12    278  -0.046124 -0.060326        1.31  outlier
13    279  -0.046124 -0.068267        1.48  outlier


In [8]:
#ana.add_prediction()
#print(ana)

In [9]:
fig_test2_features = ana.plot_data()
fig_test2_features.write_image("dev/figures/fig_test2_features.png", width=1100, height= 350, scale = 3)
fig_test2_features.show()